In [1]:
using MultivariatePolynomials
using DynamicPolynomials
using MAT
using EigenvalueSolver;
include("denseTNF.jl")

solveDense (generic function with 1 method)

In [7]:
@polyvar x[1:3];
ds = [10,10,10];
f = EigenvalueSolver.getRandomSystem_dense(x,ds);

In [8]:
function rjd_sol(M)
    size(M);
    δ = size(M[1],1)
    n = length(M);
    solMatrix = fill(0.0+0.0*im,δ,n)
    M_mu = sum(randn(n).*M);
    X = eigvecs(M_mu);
    for i = 1:n
        Uᵢ = X \ (M[i]*X)
        solMatrix[:,i] = diag(Uᵢ)
    end
    sol_rjd = map(i->solMatrix[i,:],1:δ);
    return sol_rjd
end

rjd_sol (generic function with 1 method)

In [9]:
sol_QR,M = solveDense(f,x);

constructing resultant map...
  4.304896 seconds (6.37 M allocations: 630.348 MiB, 6.77% gc time)
res is a matrix of size (4495, 3990)
computing cokernel...
 23.141253 seconds (14 allocations: 811.610 MiB, 0.04% gc time)
N has size (1000, 4495)
finding a basis using QR-P and computing multiplication matrices...
  1.141439 seconds (36.05 k allocations: 204.673 MiB, 0.37% gc time)
simultaneous diagonalization using Schur factorization...
  1.414455 seconds (1.07 k allocations: 252.284 MiB, 0.10% gc time)


In [10]:
#file = matopen("commuting_matrics_from_polynomials.mat","r")
#M = read(file, "M")
repeats = 100;
error_schur = 0;
error_rjd = 0
error_schur_c = 0;
for i=1:repeats 

    sol_rjd = rjd_sol(M);
    sol_schur = simulDiag_schur(M);
    BWEs = EigenvalueSolver.get_residual(f,sol_schur,x)
    error_schur += maximum(BWEs)
    BWEs = EigenvalueSolver.get_residual(f,sol_rjd,x)
    error_rjd += maximum(BWEs)
    sol_schur_c = simulDiag_schur(M;clustering=true)
    BWEs = EigenvalueSolver.get_residual(f,sol_schur_c,x)
    error_schur_c+=maximum(BWEs)
end
println("rSchur: ", error_schur / repeats);
println("rjd: ", error_rjd / repeats);
println("rSchur with Clusering: ", error_schur_c / repeats);

rSchur: 1.3139372074328442e-11
rjd: 9.721078754309529e-14
rSchur with Clusering: 0.009790897018065185


In [11]:
sol_fixed_schu = simulDiag_schur(M;random = false)
BWEs = EigenvalueSolver.get_residual(f,sol_fixed_schu,x)
error_fixed_schur = maximum(BWEs)
sol_fixed_schu_c = simulDiag_schur(M;random = false,clustering = true)
BWEs = EigenvalueSolver.get_residual(f,sol_fixed_schu_c,x)
error_fixed_schur_c = maximum(BWEs)
println("Schur: ", error_fixed_schur);
println("SchurClu: ", error_fixed_schur_c);

Schur: 5.5798543203084585e-12
SchurClu: 5.5798543203084585e-12


In [7]:
file = matopen("commuting_matrics_from_polynomials.mat", "w")
write(file, "M", M)
close(file)

In [10]:
size(M),size(M[1])

((3,), (1000, 1000))